Atividades
===

Essas atividade são relativas a aulas de extração de atributos e selecão de atributos. 
- Link para video: https://drive.google.com/file/d/13LLQGLt7QsjKshBXgUdJ4RVOT8N252cU/view
- Link para apresentação: https://docs.google.com/presentation/d/1wctFgQe7TSlBEypZbVQqqsfrnpSgOeXPSO-mCJ6jIz4


> Lembre de criar uma `virtualenv` com os `requirements.txt` do repositório.

> Lembre de criar uma `kernel` do jupyter para seus desenvolvimentos

> **MANTENHA** essa arquivo no local padrão do repositório. Detro da pasta **feature_extraction_and_feature_selection**.

> Enviei apenas o seu notebook para correção. Não é necessário enviar nenhum outro arquivo. Siga o padrão `Atividades-<NOME-DO-ALUNO>.ipynb`

# Feature Extraction

## 1. Realizar a extração de atributos utilzando as técnicas Fourier e HOS nas bases de dados

- Dados de falhas em aerogeradores

As orientações abaixo devem ser seguidas antes de iniciar os trabalhos

> Todo os arquivos nomeados com `v000_ ... .csv` são referentes a essa base.

> A classe referente ao dado está representada por uma TAG no nome do arquivo da seguinte forma: `v000_NORMAL_ ... .csv` é o dado referente a classe de funcionamento normal do gerador. A TAG `v000_SC_LI_LVL3_ ... .csv` é a classe referente a falha tipo 1 e continua. Não se preocupem com a ordem de enumeração das classes nesse momento.

> Há 5 arquivos na pasta.

> Utilizar as colunas a `Current_R`, `Current_S`, `Current_T` para realizar extração de atributos. Perceba que cada coluna é referente a uma sinal de corrente elétrica, portanto cada um são representados vetores de dimensões $1x50000$.

> Antes de realizar a extração de atributos deve-se dividir cada sinal em 10 partes de tamanho $1x5000$. A ideia aqui é aumentar o número de amostra da base de dados por 10. Pense que cada parte é um recorte do sinal, e ao serem concatenadas retornarão o sinal original.

# Imports  

### Importing feature extraction module  

In [28]:
!git clone https://github.com/navarmn/feature_extraction_signal.git

fatal: destination path 'feature_extraction_signal' already exists and is not an empty directory.


In [29]:
from feature_extraction_signal.src import feature_extraction

### importing pandas and others tools

In [30]:
import pandas as pd
import numpy as np
import os.path
from matplotlib import pyplot as plt

### Carregue a base

In [31]:
dataset_folder = os.path.join('.', 'datasets')
DATAFILES = [
    'v000_FAULT_SC_HI_LVL2_FR6000_FG5942_L000_0,6IN_SENSORC.csv',
    'v000_FAULT_SC_LI_LVL3_FR4500_FG4365_L000_0,8IN_SENSORC.csv',
    'v000_FAULT_SC_LI_LVL3_FR6000_FG5927_L000_0,4IN_SENSORC.csv',
    'v000_NORMAL_FR4500_FG4385_L000_1,0IN_SENSORC.csv',
    'v000_NORMAL_FR6000_FG5955_L000_0,5IN_SENSORC.csv'
]

In [71]:
#single sinals for simple tests
sinal1 = pd.read_csv(os.path.join(dataset_folder, DATAFILES[0]))
sinal2 = pd.read_csv(os.path.join(dataset_folder, DATAFILES[1]))
sinal3 = pd.read_csv(os.path.join(dataset_folder, DATAFILES[2]))
sinal4 = pd.read_csv(os.path.join(dataset_folder, DATAFILES[3]))
sinal5 = pd.read_csv(os.path.join(dataset_folder, DATAFILES[4]))

'''
We are going to have an array which each element is a dataframe with the full signal
'''

signals = [pd.read_csv(os.path.join(dataset_folder, DATAFILES[x])).iloc[:, :3] for x in range(len(DATAFILES))]

### Split signals

#### This code below will split the dataframe in an array with N dataframes

In [33]:
def splitDataframe(df, slices=10):
    #collecting number of rows and columns
    df_rows, df_columns = df.shape
    
    #defining the slice
    slice = int(df_rows/slices)
    
    #defining variables to start spliting the dataframe
    prev_slice, current_slice = 0, 0
    slices_array = [] #array that will receive the slices
    for x in range(slices):
        #update the prev and current slice variables and append the slice to slices_array 
        prev_slice = current_slice
        current_slice+=slice
        slices_array.append(df.iloc[prev_slice:current_slice])
    return slices_array    

### Joining a sliced dataframe

In [34]:
def joinDataframe(array_of_dfs):
    dataframe = pd.DataFrame([])
    for df in array_of_dfs:
        dataframe = dataframe.append(df, ignore_index=True)
    return dataframe

In [77]:
'''
Now we are going to split each signal, that by now are dataframes. So at the end of the process we
will have an array which each element it is an another array with the sliced signals.

[[slice1DF, slice2DF, ..., slice10DF], ...]

'''

splited_signals = []
for signal in signals:
    splited_signals.append(splitDataframe(signal))

    
#selecting just the three fisrt columns of the splited dataframe
#array_of_dfs = [df.iloc[:, :3] for df in array_of_dfs]

,Current_R,Current_S,Current_T
0,0.366486,1.117192,-1.372104
1,0.570073,1.193545,-1.804737
2,1.028144,0.964485,-1.473900
3,-0.206103,0.989936,-1.919258
4,1.091765,0.735425,-1.550247
...,...,...,...
4995,1.155386,-1.898767,0.803788
4996,0.799109,-1.338842,0.625644
4997,0.977247,-1.949669,1.058278
4998,0.595521,-1.402470,0.854686


### a) Comparativo da média do sinal e sua distribuição

- Realize o calculo da média e desvio padrão em cada um dos sinais e exiba o resultado em um dataframe. Discuta suas conclusões.

**help**: utilize as funcões nativas do numpy ou do pandas. Lembre que o sinal é uma senoide, qual a média em uma senoide simétrica?

In [53]:
def mean_and_std(array_of_dfs, how='slices'):
    '''
    Two types of use:
    full: Use the full signal to calculate the mean and the standard deviation.
    
    slices: Use the slices to calculate the mean and the standard deviation and return the result
    of each slice as a Dataframe
    '''

    #dataframes that will be merged
    df_mean, df_std = pd.DataFrame([]),pd.DataFrame([])
    
    #dataframe that will be returned
    return_df = pd.DataFrame([])
    
    #joins the dataframe
    dataframe = joinDataframe(array_of_dfs)

    #getting columns names and changing them
    cols_names = dataframe.columns
    mean_names, std_names = ['mean_'+name for name in cols_names],['std_'+name for name in cols_names]
    
    if how == 'full':
        
        #calculating the mean and the standard deviation of the current dataframe(df)
        serie_mean, serie_std = dataframe.mean(), dataframe.std()
        
        #changing the index of the series
        serie_mean.index,serie_std.index = mean_names,std_names
        
        #returns the mean series concatenated with the std series
        return pd.concat([serie_mean, serie_std], join='outer', sort=False)

    if how == 'slices':
        #deleting dataframe variable that will not be used anymore
        del dataframe
        
        for df in array_of_dfs:

            #calculating the mean and the standard deviation of the current dataframe(df)
            serie_mean, serie_std = df.mean(), df.std()

            #changing the index of the series
            serie_mean.index,serie_std.index = mean_names,std_names

            #appending the serie_mean and the serie_std to df_mean and df_std, respectively
            df_mean, df_std = df_mean.append(serie_mean, ignore_index=True), df_std.append(serie_std, ignore_index=True, sort=False)
    
        return_df = df_mean.merge(df_std, left_index=True, right_index=True, how='outer')
        
            
    return return_df

answer = mean_and_std(array_of_dfs, how='slices')
display(answer)



,mean_Current_R,mean_Current_S,mean_Current_T,std_Current_R,std_Current_S,std_Current_T
0,-0.009033,0.009121,0.001576,1.557126,1.293221,1.119889
1,-0.024040,0.018686,0.003087,1.548669,1.285848,1.106367
2,-0.004320,0.013443,-0.004654,1.568302,1.296482,1.104084
3,-0.006066,0.005207,0.003678,1.567200,1.305422,1.099161
4,-0.018729,0.015649,0.007681,1.561267,1.290381,1.129608
5,-0.004880,0.017815,-0.005135,1.550151,1.284158,1.114267
6,-0.007053,0.009974,-0.003980,1.558204,1.287793,1.107035
7,-0.005135,0.008126,0.005938,1.567012,1.298340,1.103462
8,-0.019887,0.018602,0.003568,1.572879,1.299555,1.104010
9,-0.008970,0.016138,0.001008,1.559266,1.293978,1.121942


È possível notar que temos a média do sinal por volta de zero por causa de sua característica periódica.

### b) Extração de atributos no sinal

- Utilize o código de `feature_extraction_signal` em: https://github.com/navarmn/feature_extraction_signal
- Use as classes `Fourier` e `HOS` de maneira adequada. Lembre do método `.transform()`.
- Na classe `Fourier`utilize os parâmetros: `fs=5000`; o valor de fundamental está na TAG do nome em `_FG5955_`significa `fundamental=59.55`. `harmonics=(0.5, 1, 1.5, 2.5, 3, 5, 7)`, mas sintam-se à vontade para buscar outras harmônicas;
- Os rótulos devem serem impostos de acordo com a descrição da tag feita cima. 
- Monte um dataframe que contenham os atributos em cada coluna e a útlima com o rótulo. Concatene os artibutos extraídos da `Current_R`, `Current_S` e `Current_T`.
- Deverá ser feito um dataframe para os atributos de Fourier e um para os atributos de HOS.

In [38]:
# code here

### c) Análise exploratória na base criada com os extratores de atributos
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.
- Faça uma análise para cada uma das bases:
    1. Features criada com Fourier
    2. Features criada com HOS
    3. Combinação das duas features (Fourier + HOS). 
> Dica: dê atenção ao coeficiente de correlação.

In [39]:
# code here

# Feature Selection

## 1. Realizar um estudo de relevância de atributos utilzando as técnicas exibidas na aula.

Nesse momento utilize a base de dados `Clinical_data_09-09-19-processed.csv`. Uma breve descrição da base:
- Contém um histórico dos registros de pacientes acometidos com uma determinada patologia.
- Os registros são variados, vão desde resposta de questionários médicos de anamnese, por exemplo, "*É fumante nos últimos 5 anos?*", "*Faz uso de álcool constantemente?*", até resultados de exames clínicos como ECG e Ecocardiograma.
- O rótulo dessa base é a coluna `Óbito`.

### Carregue a base

In [40]:
# code here

### a) Análise exploratória na base de dados
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.

In [41]:
# code here

### b) Identicar qual a variáveis parecem influenciar no óbito dos pacientes :(
- Utilize o código `feature_selection_framework` https://github.com/navarmn/feature_selection_framework 
- Monte o resultado em um `DataFrame`, as colunas serão referentes aos métodos e as linhas deverão ser os atributos. Coloque 1 quando o método indicar como relevante e 0 quando o método indicar com não relevante;
- Utilize o seguinte critério para validar um atributo como relevante: moda >= 3

In [42]:
# code here

## 2. QUESTÃO DESAFIO
- Essa etapa não entrará com atividade avaliativa, pois a base de dados a seguir não é tão bonita quanto parece 🤪. Há muitas etapas de pré-processamento, agrupmaento e mais importante de tudo, o usuário tem que fazer as duas tabelas `weekly-infos-before-shrink.csv` e `user-status-after-shrink.csv` conversarem entre si.
- Fique à vontade para tentar e para tirar dúvidas. 😊

Nesse momento utilize a base de dados `weekly-infos-before-shrink.csv` e `user-status-after-shrink.csv`. Uma breve descrição da base:
- Essa base foi retirada de um serviço de streaming de midia. Os atributos são relativos a perfis de consumo de usuários.
- O objetivo é realizar detecção de *churn* (https://resultadosdigitais.com.br/blog/o-que-e-churn/)
- Possui um registro de 17 semanas de uso e os rótulos estáo na tabela `user-status-after-shrink.csv`, que indicam se ao final do período o usuário cancelou e se manteve assinante.

In [43]:
# code here

### a) Análise exploratória na base de dados
- Use a ferramenta `pandas-profiling` https://github.com/pandas-profiling/pandas-profiling
- Faça um resumo do relatório, destacando os principais pontos gerados na base.
> Exiba a matriz de correlação!

In [44]:
# code here

### b) Identicar qual a variáveis parecem influenciar no cancelamento da assinatura.

- Utilize o código `feature_selection_framework` https://github.com/navarmn/feature_selection_framework 
- Monte o resultado em um `DataFrame`, as colunas serão referentes aos métodos e as linhas deverão ser os atributos. Coloque 1 quando o método indicar como relevante e 0 quando o método indicar com não relevante;
- Utilize o seguinte critério para validar um atributo como relevante: moda >= 3

In [45]:
# code here